<a href="https://colab.research.google.com/github/mars241/PYTHON/blob/main/AAM-WCS-2.2%20ML%20Classification%20V2%20-%20KNN%20distances%20with%20Pokemons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# 1. Importons le dataset

In [3]:
df_pokemon = pd.read_csv('https://raw.githubusercontent.com/murpi/wilddata/master/pokemon.csv')
df_pokemon.head()

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


In [4]:
df_pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   HP          800 non-null    int64 
 5   Attack      800 non-null    int64 
 6   Defense     800 non-null    int64 
 7   Sp. Atk     800 non-null    int64 
 8   Sp. Def     800 non-null    int64 
 9   Speed       800 non-null    int64 
 10  Generation  800 non-null    int64 
 11  Legendary   800 non-null    bool  
dtypes: bool(1), int64(8), object(3)
memory usage: 69.7+ KB


In [6]:
df_pokemon.isna().sum()

#               0
Name            0
Type 1          0
Type 2        386
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

# 2. Trouver les pokémons les plus proches

## 2.1 Préparer les données

In [ ]:
# Nous avons remarqué qu'il y a des valeurs manquantes, nous ne serons pas en mesure d'identifier ces Type 2 donc nous pouvons le supprimer
df_pokemon.drop("Type 2", axis=1, inplace=True)
df_pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   HP          800 non-null    int64 
 4   Attack      800 non-null    int64 
 5   Defense     800 non-null    int64 
 6   Sp. Atk     800 non-null    int64 
 7   Sp. Def     800 non-null    int64 
 8   Speed       800 non-null    int64 
 9   Generation  800 non-null    int64 
 10  Legendary   800 non-null    bool  
dtypes: bool(1), int64(8), object(2)
memory usage: 63.4+ KB


## 2.2 Trouver des pokémons non légendaires (X)

In [7]:
# Nous créons un nouveau df sans les Pokemon légendaires (nous ne pouvons pas les proposer au champion)
# Les recommandations proviendront de ce df, notre modèle KNN sera ajusté à ce df.
df_pokemon_nonlegendary = df_pokemon.loc[df_pokemon["Legendary"] == False]

In [8]:
df_pokemon_nonlegendary

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
787,788,Gourgeist Super Size,Ghost,Grass,85,100,122,58,75,54,6,False
788,789,Bergmite,Ice,NaN,55,69,85,32,35,28,6,False
789,790,Avalugg,Ice,NaN,95,117,184,44,46,28,6,False
790,791,Noibat,Flying,Dragon,40,30,35,45,40,55,6,False


In [9]:
# Ne garder que les variables numériques, supprimer la colonne # qui n'a aucune utilité en tant que variable explicative.
X = df_pokemon_nonlegendary.select_dtypes('number').drop(columns = "#")
X

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
0,45,49,49,65,65,45,1
1,60,62,63,80,80,60,1
2,80,82,83,100,100,80,1
3,80,100,123,122,120,80,1
4,39,52,43,60,50,65,1
...,...,...,...,...,...,...,...
787,85,100,122,58,75,54,6
788,55,69,85,32,35,28,6
789,95,117,184,44,46,28,6
790,40,30,35,45,40,55,6


## 2.3 Ajuster le modèle

In [10]:
# Ajuster le modèle avec 3 voisins
# notez que ce modèle est entraîné avec X pour lequel les variables proviennent de df_pokemon_nonlegendary
modelNN = NearestNeighbors(n_neighbors=3)
modelNN.fit(X)

NearestNeighbors(n_neighbors=3)

## 2.4 Trouver les pokémons des champions

In [11]:
# Faire une liste des Pokemons du champion
df_pokemon.loc[df_pokemon['Name'].str.contains('Giratina')]  # Attention : Giratina a deux noms différents.

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
544,545,Giratina Altered Forme,Ghost,Dragon,150,100,120,100,120,90,4,True
545,546,Giratina Origin Forme,Ghost,Dragon,150,120,100,120,100,90,4,True


In [12]:
list_champion = ['Mewtwo','Lugia','Rayquaza','Giratina Origin Forme', 'Dialga', 'Palkia']

## 2.5 Trouver les pokémons les plus proches

In [13]:
# Pour chaque Pokemon de la liste trouver 3 voisins les plus proches
for Pokemon in list_champion:
    # Pour un Pokemon, passer seulement les mêmes colonnes que X dont notre modèle a besoin
    neighbors = modelNN.kneighbors(df_pokemon.loc[df_pokemon['Name'] == Pokemon, X.columns])
    print(f"Recommandations de(s) Pokemon(s) {Pokemon} :")
    # trouver le numéro de ligne (et non le nom de la ligne) des plus proches voisins dans le cadre de données sur lequel le modèle a été ajusté.
    closest_pok_ind = neighbors[1][0]
    closest_pok = df_pokemon_nonlegendary['Name'].iloc[closest_pok_ind] # ! Avertissement ! nous pouvons utiliser df_pokemon_nonlegendary car X est basé sur ce dataframe
    print("Pokemon les plus proches : ", list(closest_pok))
    print("Respectives distances : ", neighbors[0][0])
    print()

Recommandations de(s) Pokemon(s) Mewtwo :
Pokemon les plus proches :  ['Mega Houndoom', 'Mega Sceptile', 'Mega Pidgeot']
Respectives distances :  [42.22558466 43.08131846 45.50824101]

Recommandations de(s) Pokemon(s) Lugia :
Pokemon les plus proches :  ['Cresselia', 'Mega Venusaur', 'Celebi']
Respectives distances :  [46.10856753 62.49799997 64.43601477]

Recommandations de(s) Pokemon(s) Rayquaza :
Pokemon les plus proches :  ['Mega Blaziken', 'Mega Garchomp', 'Mega Charizard X']
Respectives distances :  [36.74234614 44.37341546 44.98888752]

Recommandations de(s) Pokemon(s) Giratina Origin Forme :
Pokemon les plus proches :  ['Manaphy', 'Celebi', 'Mew']
Respectives distances :  [58.30951895 58.34380858 58.38664231]

Recommandations de(s) Pokemon(s) Dialga :
Pokemon les plus proches :  ['Mega Blastoise', 'Mega Charizard X', 'Hydreigon']
Respectives distances :  [36.51027253 37.40320842 44.48595284]

Recommandations de(s) Pokemon(s) Palkia :
Pokemon les plus proches :  ['Mega Charizard